<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-deeplearning/blob/master/jyp/skip_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class skip_gram:
    def __init__(self,docs_ls):
        self.docs_ls=docs_ls
        self.word_ls=sorted(list(set(sum(self.docs_ls,[]))))

    def connected_token(self,token,window_size):
        token_index_ls=[[i for i,v in enumerate(doc) if v==token] for doc in docs_ls]
        return_set={self.word_ls.index(v) for k,_ in enumerate(self.docs_ls) for i,v in enumerate(self.docs_ls[k]) for j in token_index_ls[k] if abs(i-j)<=window_size}-{self.word_ls.index(token)}
        return sorted(list(return_set))

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk import pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\student\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
with open("./text/AA/wiki_00",mode="r",encoding="utf-8") as f:
    text=f.read()
textdf=pd.DataFrame(text.split("</doc>")[:-1])
textdf.columns=["content"]
textdf["content"]=textdf["content"].str.replace('<.*?>',"").str.replace("\n"," ")
textdf["tokenize"]=textdf["content"].apply(word_tokenize)
textdf["pos_tag"]=textdf["tokenize"].apply(pos_tag)

pos=["JJ","JJR", "JJS","NN","NNS","NNP","NNPS","VB","VBD","VBG","VBN","VBP","VBZ"]

textdf["reduced_postag"]=textdf["pos_tag"].apply(lambda ls: [v for v in ls if (v[1] in pos) and (len(v[0])>1)])
textdf["token_ls"]=textdf["reduced_postag"].apply(lambda ls: [v[0] for v in ls])

df=pd.DataFrame([",".join(token_ls) for token_ls in textdf["token_ls"].tolist()])

In [ ]:
docs_ls=df[0].str.split(",").apply(lambda x :x[1:]).tolist()
docs_ls

[['Anarchism',
  'is',
  'political',
  'philosophy',
  'movement',
  'rejects',
  'involuntary',
  'coercive',
  'forms',
  'hierarchy',
  'calls',
  'abolition',
  'state',
  'holds',
  'be',
  'undesirable',
  'unnecessary',
  'harmful',
  'history',
  'anarchism',
  'goes',
  'prehistory',
  'humans',
  'lived',
  'anarchistic',
  'societies',
  'establishment',
  'formal',
  'states',
  'realms',
  'empires',
  'rise',
  'organised',
  'hierarchical',
  'bodies',
  'skepticism',
  'authority',
  'rose',
  'was',
  '19th',
  'century',
  'self-conscious',
  'political',
  'movement',
  'emerged',
  'latter',
  'half',
  'first',
  'decades',
  '20th',
  'century',
  'anarchist',
  'movement',
  'flourished',
  'most',
  'parts',
  'world',
  'had',
  'significant',
  'role',
  'workers',
  'struggles',
  'emancipation',
  'Various',
  'anarchist',
  'schools',
  'thought',
  'formed',
  'period',
  'Anarchists',
  'have',
  'taken',
  'part',
  'several',
  'revolutions',
  'most',

In [ ]:
from collections import Counter
fre_ls=np.array([Counter(sum(docs_ls,[]))[word] for word in skip_gram(docs_ls).word_ls])
distribution=np.power(fre_ls,0.75)/np.power(fre_ls,0.75).sum()
n=len(skip_gram(docs_ls).word_ls)
negative_words=np.random.choice(range(n), size=20, p=distribution)


In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
n

18816

In [ ]:
word_ls=skip_gram(docs_ls).word_ls
hidden_num=3
batch_size=100
learning_rate=0.001
num_epoch=10

W_1=np.random.rand(n,hidden_num)
W_2=np.random.rand(n,hidden_num)
for epoch in range(num_epoch):
    for i in range(-round(-n/batch_size)):
        for j in range(i*batch_size,min((i+1)*batch_size,n)):
            neighbor=skip_gram(docs_ls).connected_token(word_ls[j],1)

            positive=np.ones(len(neighbor))
            negative=(negative_words.reshape(-1,1)==np.array(neighbor)).sum(axis=1)

            y=np.concatenate((positive,negative))
            hyp=sigmoid(np.dot(W_2[neighbor+list(negative_words)],W_1[j]))
            loss=-(y*np.log(hyp)+(1-y)*np.log(1-hyp)).mean()

            first_par=W_1[j]
            W_1-=np.dot(W_2[neighbor+list(negative_words)].T,hyp-y)*learning_rate/len(y)
            W_2[neighbor+list(negative_words)]-=np.dot((hyp-y).reshape(-1,1),first_par.reshape(1,-1))*learning_rate/len(y)

In [ ]:
W_1

array([[-0.20590584,  3.60326714, -2.7750958 ],
       [ 0.18165076,  3.48983398, -2.46369364],
       [-0.46861339,  3.32133333, -2.54813176],
       ...,
       [-0.12345834,  3.22972105, -2.3198024 ],
       [-0.2601894 ,  3.55670432, -2.76479253],
       [ 0.14113007,  2.96076656, -2.71294743]])

In [ ]:
word1_index=word_ls.index("Artists")
word2_index=word_ls.index("Bell")
result_vec=W_1[word1_index]-W_1[word2_index]
word_ls[np.argmin(np.multiply(W_1-result_vec,W_1-result_vec).sum(axis=1))]

'lexicostatistical'